## _*Monty Hall*_ 

The latest version of this notebook is available on https://github.com/QISKit/qiskit-tutorial.

For more information about how to use the IBM Q experience (QX), consult the [tutorials](https://quantumexperience.ng.bluemix.net/qstage/#/tutorial?sectionId=c59b3710b928891a1420190148a72cce&pageIndex=0), or check out the [community](https://quantumexperience.ng.bluemix.net/qstage/#/community).

***
### Contributors


In [42]:
# Checking the version of PYTHON; we only support 3 at the moment
import sys
if sys.version_info < (3,0):
    raise Exception('Please use Python version 3 or greater.')
    
# useful additional packages 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import time

import sys
sys.path.append('C:/qiskit-sdk-py')
sys.path.append('C:/qiskit-sdk-py/qiskit')
# importing the QISKit
from qiskit import QuantumCircuit, QuantumProgram
import Qconfig

# import basic plot tools
from tools.visualization import plot_histogram

In [43]:
QPS_SPECS = {
    'circuits': [{
        'name': 'monty',
        'quantum_registers': [{
            'name':'q',
            'size':5
        }],
        'classical_registers': [{
            'name':'c',
            'size':5
        }]}],
}



In [44]:
"SETTING THE CIRCUIT"

Q_program = QuantumProgram(specs=QPS_SPECS)
Q_program.set_api(Qconfig.APItoken, Qconfig.config['url'])
monty = Q_program.get_circuit('monty') 
q = Q_program.get_quantum_register('q') 
c = Q_program.get_classical_register('c')

monty.ry(1.911,q[2]);
monty.cx(q[2],q[1]);
monty.ry(-np.pi/4,q[2]);
monty.h(q[1]);
monty.h(q[2]);
monty.cx(q[2],q[1]);
monty.h(q[1]);
monty.h(q[2]);
monty.ry(np.pi/4,q[2])
monty.x(q[0]);
monty.cx(q[2],q[1]);
monty.cx(q[1],q[0]);
monty.cx(q[2],q[0]);
monty.h(q[3]);
monty.h(q[4]);
monty.measure(q[0] , c[0]);
monty.measure(q[1] , c[1]);
monty.measure(q[2] , c[2]);
monty.measure(q[3] , c[3]);
monty.measure(q[4] , c[4]);

circuits = ['monty']
print('backend: make your choice by running one, and only one, of the TWO next cells')

backend: make your choice by running one, and only one, of the TWO next cells


In [45]:
"REAL QUANTUM COMPUTER!"
backend = 'ibmqx4'

In [46]:
"SIMULATOR"
backend = 'local_qasm_simulator'

In [77]:
"HERE START THE GAME"
"Hiding the Cadillac and the two goats behind the three doors"
shots = 1
nb_of_cadillacs = 4

while nb_of_cadillacs != 1:
    result = Q_program.execute(circuits, backend=backend, shots=shots, max_credits=5, wait=5, timeout=600, silent=False)
    time_exp = time.strftime('%d/%m/%Y %H:%M:%S')
    c5str = str(result.get_counts('monty'))
    nb_of_cadillacs = int(c5str[4]) + int(c5str[5]) + int(c5str[6])
    if nb_of_cadillacs == 0:
        print("They managed to hide three goats and no cadillac behind the doors! Restart of the hiding process...")
    if nb_of_cadillacs >= 2: 
        print("They managed to hide", nb_of_cadillacs, "cadillacs behind the doors! Restart of the hiding process...")      
print('Game Master: One cadillac and two goats are now hidden behind these doors.')
print('             Please make your choice by running one, and only one, of the THREE following cells.')

running on backend: local_qasm_simulator
Game Master: One cadillac and two goats are now hidden behind these doors.
             Please make your choice by running one, and only one, of the THREE following cells.


In [78]:
"Chosing the door that you see to your left"
Doorchosen = 1

In [79]:
"Chosing the door that you see in front of you"
Doorchosen = 2

In [80]:
"Chosing the door that you see to your right"
Doorchosen = 3

In [81]:
"AN OPPORTUNITY TO CHANGE YOUR MIND"

Label = ["left", "central", "right"]
c5str = str(result.get_counts('monty'))

randomnb = int(c5str[2]+c5str[3],2) #OK
if randomnb > 1:
    randomnb = randomnb - 2
    
if c5str[4] == "1":    
    if Doorchosen == 1:
        Dooropen = 2 + randomnb
        Doorswitch = 3 - randomnb
    if Doorchosen == 2:
        Dooropen = 3
        Doorswitch = 1
    if Doorchosen == 3:
        Dooropen = 2
        Doorswitch = 1      
               
if c5str[5] == "1":
    if Doorchosen == 2:
        Dooropen = 1 + 2*randomnb
        Doorswitch = 3 - 2*randomnb
    if Doorchosen == 1: 
        Dooropen = 3
        Doorswitch = 2
    if Doorchosen == 3:   
        Dooropen = 1
        Doorswitch = 2
        
if c5str[6] == "1":
    if Doorchosen == 3:
        Dooropen = randomnb + 1
        Doorswitch = 2 - randomnb
    if Doorchosen == 1: 
        Dooropen = 2
        Doorswitch = 3
    if Doorchosen == 2:   
        Dooropen = 1
        Doorswitch = 3
            
print('Game Master: I opened the', Label[Dooropen-1], 'door and you see a goat ')
print(' Your choice was the',Label[Doorchosen-1], "door")
print(' You get now an opportunity to chance your choice!')
print(' Do you want to switch for the',Label[Doorswitch-1], "door?"' ')
print('             Please make your choice by running one, and only one, of the TWO following cells.')

Game Master: I opened the left door and you see a goat 
 Your choice was the right door
 You get now an opportunity to chance your choice!
 Do you want to switch for the central door? 
             Please make your choice by running one, and only one, of the TWO following cells.


In [91]:
"I STICK TO MY FIRST CHOICE!"
Doorfinal = Doorchosen

In [93]:
"I CHANGE MY MIND!"
Doorfinal = Doorswitch     

In [94]:
"FINAL ANNOUNCE"
if c5str[Doorfinal+3] == "1":
    endmessage = 'won the Cadillac! Congratulations!'
else:
    endmessage = 'won a goat! Sorry!'
print('Game Master: You opened the',Label[Doorfinal-1],'door and', endmessage)

Game Master: You opened the central door and won the Cadillac! Congratulations!


In [95]:
print(time_exp, backend, c5str, Doorchosen, Doorfinal) #check

30/10/2017 10:02:37 local_qasm_simulator {'10010': 1} 3 2
